In [1]:
import os
import argparse
import numpy as np
import pandas as pd
import time

import torch
from torch import optim
import torch.nn as nn

from torch.utils.data import DataLoader
import torch.nn.functional as F

import torchvision.datasets as datasets
from torchvision import transforms
from data_loader import *
from model import *
from query import *

In [2]:
DEFAULT_DATASET_DIR = "./dataset/"
DEFAULT_MODEL_PATH = "./ckpt/exp_13/model_ckpt.tar"

In [3]:
images_folder_dataset = datasets.ImageFolder(root=DEFAULT_DATASET_DIR+'signatures/valid')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset)
print(len(siamese_dataset))

144


In [4]:
query_model = QueryModel(DEFAULT_MODEL_PATH)

D:\Codes\axis-ai-challenge\model\model.py:73: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_in')


In [17]:
img0_file = siamese_dataset.genuine[0][1]
img1_file = siamese_dataset.genuine[1][1]
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('./dataset/signatures/valid\\full_org\\original_50_1.png',
 './dataset/signatures/valid\\full_org\\original_50_10.png')

In [ ]:
genuine_sigs = siamese_dataset.genuine
forged_sigs = siamese_dataset.forged

In [ ]:
len(siamese_dataset)

In [ ]:
output = query_model.predict(img0, img1)

In [ ]:
print(output.item())

In [ ]:
img0_file = siamese_dataset.genuine[63][1]
img1_file = siamese_dataset.forged[63][1]
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

In [ ]:
output = query_model.predict(img0, img1)

In [ ]:
print(output.item())

In [11]:
img0_file = DEFAULT_DATASET_DIR+'DataBase/U11001_79.jpg'
img1_file = DEFAULT_DATASET_DIR+'DataBase/U11001_152.jpg'
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('./dataset/DataBase/U11001_79.jpg', './dataset/DataBase/U11001_152.jpg')

In [12]:
output = query_model.predict(img0, img1)

In [13]:
print(output.item())

0.9254697561264038


# CEDAR Dataset Scores

In [41]:
images_folder_dataset = datasets.ImageFolder(root=DEFAULT_DATASET_DIR+'signatures/valid')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset)

In [42]:
len(siamese_dataset)

144

In [43]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0
# for i in range(1, 10):
for x1, x2, label in siamese_dataset:
    print(label)
    print()
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.getConfidence(x1, x2)
    print(output.item())

    if output >= 0.5 and label == 1:
        tp += 1
        print("TRUE POSITIVE")
    elif output <= 0.5 and label == 0:
        tn += 1
        print("TRUE NEGATIVE")
    elif output >= 0.5 and label == 0:
        fp += 1
        print("FALSE POSITIVE")
    elif output <= 0.5 and label == 1:
        fn += 1
        print("FALSE NEGATIVE")

    print()

tensor([ 0.])



C:\Users\arush\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


0.004360708873718977
TRUE NEGATIVE

tensor([ 0.])

0.06657037138938904
TRUE NEGATIVE

tensor([ 0.])

0.01859249733388424
TRUE NEGATIVE

tensor([ 1.])

0.4409199655056
FALSE NEGATIVE

tensor([ 1.])

0.9374963641166687
TRUE POSITIVE

tensor([ 0.])

0.12402660399675369
TRUE NEGATIVE

tensor([ 1.])

0.6952945590019226
TRUE POSITIVE

tensor([ 0.])

0.0014478318626061082
TRUE NEGATIVE

tensor([ 0.])

0.04940718412399292
TRUE NEGATIVE

tensor([ 0.])

0.06973029673099518
TRUE NEGATIVE

tensor([ 0.])

0.002176983980461955
TRUE NEGATIVE

tensor([ 1.])

0.6418536901473999
TRUE POSITIVE

tensor([ 1.])

0.4969501495361328
FALSE NEGATIVE

tensor([ 1.])

0.24469909071922302
FALSE NEGATIVE

tensor([ 1.])

0.6901801824569702
TRUE POSITIVE

tensor([ 0.])

0.002255768748000264
TRUE NEGATIVE

tensor([ 0.])

0.058123212307691574
TRUE NEGATIVE

tensor([ 1.])

0.6216535568237305
TRUE POSITIVE

tensor([ 0.])

0.04403992369771004
TRUE NEGATIVE

tensor([ 1.])

0.8469480276107788
TRUE POSITIVE

tensor([ 0.])

0.

In [44]:
correct = tp+tn
print(100*correct/total)

print('TN: {}'.format((tn/total)*100), 'FP: {}'.format((fp/total)*100))
print('FN: {}'.format((fn/total)*100), 'TP: {}'.format((tp/total)*100))

recall = tp / (tp+fn)
precision = tp / (tp+fp)

f1_score = 2*((precision*recall)/(precision+recall))

print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1 Score: {}'.format(f1_score))


91.66666666666667
TN: 49.30555555555556 FP: 1.3888888888888888
FN: 6.944444444444445 TP: 42.36111111111111
Precision: 0.9682539682539683
Recall: 0.8591549295774648
F1 Score: 0.9104477611940299


# Sample Dataset from Axis AI Scores

In [45]:
images_folder_dataset = datasets.ImageFolder(root=DEFAULT_DATASET_DIR+'dataset3/train')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset, dataset_type='sample')

In [46]:
len(siamese_dataset)

140

In [47]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0
# for i in range(1, 10):
for x1, x2, label in siamese_dataset:
    print(label)
    print()
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.getConfidence(x1, x2)
    print(output.item())

    if output >= 0.5 and label == 1:
        tp += 1
        print("TRUE POSITIVE")
    elif output <= 0.5 and label == 0:
        tn += 1
        print("TRUE NEGATIVE")
    elif output >= 0.5 and label == 0:
        fp += 1
        print("FALSE POSITIVE")
    elif output <= 0.5 and label == 1:
        fn += 1
        print("FALSE NEGATIVE")

    print()

tensor([ 1.])



C:\Users\arush\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


0.9038991332054138
TRUE POSITIVE

tensor([ 0.])

0.00011348829139024019
TRUE NEGATIVE

tensor([ 1.])

0.5181291103363037
TRUE POSITIVE

tensor([ 1.])

0.7039604187011719
TRUE POSITIVE

tensor([ 0.])

0.0003542755439411849
TRUE NEGATIVE

tensor([ 1.])

0.935896635055542
TRUE POSITIVE

tensor([ 1.])

0.9908332228660583
TRUE POSITIVE

tensor([ 1.])

0.5181291103363037
TRUE POSITIVE

tensor([ 1.])

0.9959856867790222
TRUE POSITIVE

tensor([ 0.])

0.1394502818584442
TRUE NEGATIVE

tensor([ 1.])

0.18658873438835144
FALSE NEGATIVE

tensor([ 1.])

0.6382030844688416
TRUE POSITIVE

tensor([ 1.])

0.9495256543159485
TRUE POSITIVE

tensor([ 1.])

0.9277262091636658
TRUE POSITIVE

tensor([ 0.])

0.9577612280845642
FALSE POSITIVE

tensor([ 0.])

1.672756752668647e-06
TRUE NEGATIVE

tensor([ 1.])

0.8128364086151123
TRUE POSITIVE

tensor([ 0.])

0.924127459526062
FALSE POSITIVE

tensor([ 0.])

2.9166716331019416e-07
TRUE NEGATIVE

tensor([ 0.])

0.0048409923911094666
TRUE NEGATIVE

tensor([ 1.])

0

In [60]:
correct = tp+tn
print('Accuracy: {}'.format(100*correct/total))

print('TN: {}'.format((tn/total)*100), 'FP: {}'.format((fp/total)*100))
print('FN: {}'.format((fn/total)*100), 'TP: {}'.format((tp/total)*100))

recall = tp / (tp+fn)
precision = tp / (tp+fp)

f1_score = 2*((precision*recall)/(precision+recall))

print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1 Score: {}'.format(f1_score))



Accuracy: 98.55442176870748
TN: 51.44557823129252 FP: 0.8503401360544218
FN: 0.5952380952380952 TP: 47.10884353741497
Precision: 0.9822695035460993
Recall: 0.9875222816399287
F1 Score: 0.9848888888888889
